# Analyse des erreurs de prédictions

## Introduction

Ce notebook contient un bout de code permettant d'afficher les mauvaises prédictions de nos modèles de classifications. Il affiche les différents panneaux de signalisation mal prédits par le modèle, avec la prédiction du modèle et la vraie classe du panneau.

### Etape 1
Changer le répertoire de travail en utilisant le code suivant:
```python
os.chdir(f'/home/axel/Bureau/5_Mod_IA/Semestre_1_INSA/Statistiques_Grande_Dimension_Apprentissage_Profond/Projet/Projet_HDDL_2/Commun/Commun_Axel')
```
### Etape 2
Exécute le code cellule par cellule. Le code est censé importer les modules nécéssaires, importer les données du type voulu (Dangers, Obligations, Interdictions, Indications....), importer le modèle associé, et enfin prédire les panneaux de signalisation.

## Importation des modules et définition de l'emplacement du workspace

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import sys
import os

os.chdir(f'/home/axel/Bureau/5_Mod_IA/Semestre_1_INSA/Statistiques_Grande_Dimension_Apprentissage_Profond/Projet/Projet_HDDL_2/Commun/Commun_Axel')
print(os.getcwd())
def check_path_exists(Path,Message):
    if not os.path.exists(Path):
        print(Message)
        sys.exit(1)

# Cette partie permet d'ajouter le dossier parent au path, pour pouvoir importer les modules que nous avons créés. ça peut être soit un chemin
# absolu, soit un chemin relatif. Ici, c'est un chemin absolu.
Path_Modules = "./Modules"
Path_Models = "./Models"
check_path_exists(Path_Modules,"Le chemin spécifié pour importer les modules n'existe pas. Il faut surement le modifier.")

sys.path.append(Path_Modules)
sys.path.append(Path_Models)
# Importer les module
from Preprocessing import *
from Resnet import *
from Train import *

## Définitions de fonctions utiles pour la suite

In [25]:
def load_encoding_dictionaries(path,title_label_to_int ,title_int_to_label):
    
    def check_path_exists(path,message):
        if not os.path.exists(path):
            raise ValueError(message)
        else:
            pass

    check_path_exists(path,"The specified path for the encoding dictionaries does not exist. You probably need to modify it.")

    # Load JSON into dictionaries
    with open(path+title_label_to_int+".json", "r") as read_file:
        label_to_int = json.load(read_file)

    with open(path+title_int_to_label+".json", "r") as read_file:
        int_to_label = json.load(read_file)

    return label_to_int,int_to_label




def model_labels(Type = "Indications"):
    Image_Information_Path = "./Anotations/"
    Name_Image_Information = "Image_Information.csv"
    Name_Image_Information_Train = "Image_Information_Train.csv"
    Name_Image_Information_Test = "Image_Information_Test.csv"

    # Checker si le fichier existe
    #check_path_exists(Image_Information_Path,"Le chemin spécifié pour le DataFrame n'existe pas. Il faut surement le modifier.")

    # Charger le DataFrame
    Image_Information_Dataframe = pd.read_csv(Image_Information_Path+Name_Image_Information)
    Image_Information_Dataframe_Train = pd.read_csv(Image_Information_Path+Name_Image_Information_Train)
    Image_Information_Dataframe_Test = pd.read_csv(Image_Information_Path+Name_Image_Information_Test)


    if Type == "Types": 
        pass
    else: 
        Image_Information_Dataframe_Test = Image_Information_Dataframe_Test[Image_Information_Dataframe_Test["Type"]== Type]
        Image_Information_Dataframe_Train = Image_Information_Dataframe_Train[Image_Information_Dataframe_Train["Type"]== Type]
        Image_Information_Dataframe = Image_Information_Dataframe[Image_Information_Dataframe["Type"]== Type]

    print(f"Nombre d'images dans le dataset: {Image_Information_Dataframe.shape[0]}")
    print(f"Nombre d'images dans le dataset de train: {Image_Information_Dataframe_Train.shape[0]}")
    print(f"Nombre d'images dans le dataset de test: {Image_Information_Dataframe_Test.shape[0]}")


    # Preprocessing des images
    resize=True
    size=(224,224)
    normalize=True
    # Nb_Images_To_Import = 5
    Frac_Images_To_Import = 1

    images_train, types_train, sublabels_train, labels_train = import_images_tensor(Image_Information_Dataframe_Train.sample(frac=Frac_Images_To_Import),"Relative_Path", "Type", "Sublabel", "Label",resize=resize, size=size,normalize=normalize)
    images_test , types_test, sublabels_test, labels_test = import_images_tensor(Image_Information_Dataframe_Test.sample(frac=Frac_Images_To_Import),"Relative_Path", "Type", "Sublabel", "Label",resize=resize, size=size,normalize=normalize)



    # # # Création des dictionnaires d'encodage
    save_dicts = False
    Path_Save_Dicts = f"./Encoding_Dictionaries/"

    # Load Encoding Dictionaries
    types_to_int , int_to_types = load_encoding_dictionaries(Path_Save_Dicts+"Types/","types_to_int","int_to_types")

    if Type !="Types" :
        labels_to_int , int_to_labels = load_encoding_dictionaries(Path_Save_Dicts + Type + "/","labels_to_int","int_to_labels")
        sublabels_to_int , int_to_sublabels = load_encoding_dictionaries(Path_Save_Dicts + Type + "/","sublabels_to_int","int_to_sublabels")
    else:
        sublabels_to_int , int_to_sublabels = Create_Encoding_Label_Dictionary(sublabels_train,save=save_dicts, path=Path_Save_Dicts,title_label_to_int='sublabels_to_int',title_int_to_label='int_to_sublabels')
        labels_to_int , int_to_labels = Create_Encoding_Label_Dictionary(labels_train,save=save_dicts, path=Path_Save_Dicts,title_label_to_int='labels_to_int',title_int_to_label='int_to_labels')
    
    # # Encodage des labels

    types_encoded_train,types_encoded_test = encode_labels(types_train, types_to_int) , encode_labels(types_test, types_to_int)
    sublabels_encoded_train,sublabels_encoded_test = encode_labels(sublabels_train, sublabels_to_int) , encode_labels(sublabels_test, sublabels_to_int)
    labels_encoded_train,labels_encoded_test = encode_labels(labels_train, labels_to_int) , encode_labels(labels_test, labels_to_int)


    # # Creating Dataloaders : Train and Test
    dataloader_train, _ = create_dataloader(images_train, types_encoded_train, sublabels_encoded_train, labels_encoded_train, batch_size=32, test_size=0, shuffle=True)
    dataloader_test ,_= create_dataloader(images_test, types_encoded_test, sublabels_encoded_test, labels_encoded_test, batch_size=32, test_size=0, shuffle=True)

    # # load the model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Evalutation sur {device}")
    model = torch.load(f'./Saved_Models/Model_{Type}.pth').to(device)
    model.eval()


    if Type =="Types" : 
        return model, None, None, int_to_types, dataloader_test, dataloader_train
    else:
        return model, int_to_labels, int_to_sublabels, int_to_types, dataloader_test, dataloader_train
    

# REprende tous les mauvais predictions
def get_all_false_predictions(model,dataloader,Type):
    with torch.no_grad():
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.eval()
        all_false_predictions = []
        for i, data in enumerate(dataloader):
            images, types, sublabels, labels = data
            images, types, sublabels, labels = images.to(device), types.to(device), sublabels.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            for i in range(len(predicted)):
                if Type=="Types":
                    if predicted[i] != types[i]:
                        all_false_predictions.append((images[i],predicted[i],types[i]))
        
                    # if len(all_false_predictions) == 4:
                    #     return all_false_predictions
                else:
                    if predicted[i] != labels[i]:
                        all_false_predictions.append((images[i],predicted[i],labels[i]))
        
                    # if len(all_false_predictions) == 4:
                    #     return all_false_predictions

    return all_false_predictions



# Afficher les mauvaises predictions


def show_false_prediction(int_to_labels,all_false_predictions):

    def imshow(img):
        npimg = img.cpu().numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

    # plt.subplots(len(all_false_predictions), 1, figsize=(12, 12))
    for i in range(len(all_false_predictions)):
        plt.figure(figsize=(10, 6))
        imshow(all_false_predictions[i][0])
        plt.title(f"$\\bf{{Predicted}}$: {int_to_labels[str(all_false_predictions[i][1].item())]},     "
                f"$\\bf{{True}}$: {int_to_labels[str(all_false_predictions[i][2].item())]}", fontsize=14)
        plt.axis('off')




## Analyse de l'erreur du modèle de prédiction du type de panneau( Dangers, Obligations, Interdictions, Indications....)

In [ ]:
model_Type, int_to_labels, int_to_sublabels, int_to_types, dataloader_test, dataloader_train = model_labels("Types")


In [27]:


all_false_predictions_types = get_all_false_predictions(model_Type,dataloader_test,'Types')
    

In [ ]:
show_false_prediction(int_to_labels=int_to_types,all_false_predictions=all_false_predictions_types)

## Analyse de l'erreur Dangers

In [ ]:
model_Dangers, int_to_labels_Dangers, int_to_sublabels_Dangers, int_to_types, dataloader_test, dataloader_train = model_labels("Dangers")


In [8]:
all_false_predictions_dangers = get_all_false_predictions(model_Dangers,dataloader_test,'Dangers')

In [ ]:
show_false_prediction(int_to_labels=int_to_sublabels_Dangers,all_false_predictions=all_false_predictions_dangers)

## Analyse de l'erreur Obligations

In [ ]:
model_Obligations, int_to_labels_Obligations, int_to_sublabels_Obligations, int_to_types, dataloader_test, dataloader_train = model_labels("Obligations")


In [14]:
all_false_predictions_obligations = get_all_false_predictions(model_Obligations,dataloader_test,'Obligations')

In [ ]:
show_false_prediction(int_to_labels=int_to_sublabels_Obligations,all_false_predictions=all_false_predictions_obligations)

## Analyse de l'erreur Interdictions

In [ ]:
model_Interdictions, int_to_labels_Interdictions, int_to_sublabels_Interdictions, int_to_types, dataloader_test, dataloader_train = model_labels("Interdictions")


In [17]:
all_false_predictions_obligations = get_all_false_predictions(model_Interdictions,dataloader_test,'Interdictions')

In [ ]:
show_false_prediction(int_to_labels=int_to_sublabels_Interdictions,all_false_predictions=all_false_predictions_obligations)

## Analyse de l'erreur Indications

In [ ]:
model_Indications, int_to_labels_Indications, int_to_sublabels_Indications, int_to_types, dataloader_test, dataloader_train = model_labels("Indications")


In [20]:
all_false_predictions_obligations = get_all_false_predictions(model_Indications,dataloader_test,'Indications')

In [ ]:
show_false_prediction(int_to_labels=int_to_sublabels_Indications,all_false_predictions=all_false_predictions_obligations)

## Analyse de l'erreur Fin_Interdictions

In [ ]:
model_Fin_Interdictions, int_to_labels_Fin_Interdictions, int_to_sublabels_Fin_Interdictions, int_to_types, dataloader_test, dataloader_train = model_labels("Fin_Interdictions")


In [23]:
all_false_predictions_obligations = get_all_false_predictions(model_Fin_Interdictions,dataloader_test,'Fin_Interdictions')

In [ ]:
show_false_prediction(int_to_labels=int_to_sublabels_Fin_Interdictions,all_false_predictions=all_false_predictions_obligations)